In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import norm, t

# Black-Scholes option pricing
def black_scholes(S, K, r, T, sigma, option_type="call"):
    """
    Black-Scholes formula for European call and put options.

    Parameters:
        S (float): Current asset price
        K (float): Strike price
        r (float): Risk-free interest rate
        T (float): Time to maturity (in years)
        sigma (float): Volatility (standard deviation of log returns)
        option_type (str): "call" or "put"

    Returns:
        float: Option price
    """
    d1 = (np.log(S / K) + (r + sigma**2 / 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if option_type == "call":
        price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif option_type == "put":
        price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    else:
        raise ValueError("Invalid option_type. Choose 'call' or 'put'.")

    return price

# Pricing functions for the instruments
def straddle_price(S, K, r, T, sigma):
    """
    Price a straddle (call + put) under Gaussian assumptions.
    """
    call_price = black_scholes(S, K, r, T, sigma, option_type="call")
    put_price = black_scholes(S, K, r, T, sigma, option_type="put")
    return call_price + put_price

def protective_put_price(S, K, r, T, sigma):
    """
    Price a protective put (asset price + put option).
    """
    put_price = black_scholes(S, K, r, T, sigma, option_type="put")
    return S + put_price

def calculate_prices(df):
    """
    Calculate the current asset price, straddle price, and protective put price.

    Parameters:
        df (pd.DataFrame): DataFrame where each column represents 1 year of daily asset returns.

    Returns:
        pd.DataFrame: DataFrame with columns for calculated prices.
    """
    initial_price = 100
    r = 0  # Risk-free rate
    T = 1  # Time horizon in years

    results = []

    for col in df.columns:
        # Calculate current price
        log_returns = df[col].dropna()
        current_price = initial_price * np.exp(log_returns.cumsum().iloc[-1])

        # Calculate realized volatility
        sigma = log_returns.std() * np.sqrt(252)  # Annualized volatility

        # Straddle price (strike = current price)
        straddle = straddle_price(current_price, current_price, r, T, sigma)

        # Protective put price (strike = 2 sigma below current price)
        protective_put_strike = current_price - 2 * sigma * current_price
        protective_put = protective_put_price(current_price, protective_put_strike, r, T, sigma)

        # Append results
        results.append({
            "Column": col,
            "Current Price": current_price,
            "Realized Volatility": sigma,
            "Straddle Price": straddle,
            "Protective Put Price": protective_put
        })

    return pd.DataFrame(results)

def evaluate_positions(test_df, prices_df):
    """
    Evaluate the returns of straddle, protective put, and buy-and-hold positions on test data.

    Parameters:
        test_df (pd.DataFrame): Test dataset where each column represents daily log returns.
        prices_df (pd.DataFrame): DataFrame with calculated prices from `calculate_prices`.

    Returns:
        pd.DataFrame: DataFrame with returns for each position on each asset.
    """
    initial_price = 100

    results = []

    for col in test_df.columns:
        # Get test log returns
        log_returns = test_df[col].dropna()
        final_price = initial_price * np.exp(log_returns.cumsum().iloc[-1])

        # Retrieve calculated prices for the asset
        row = prices_df[prices_df["Column"] == col].iloc[0]
        current_price = row["Current Price"]
        straddle_price = row["Straddle Price"]
        protective_put_price = row["Protective Put Price"]

        # Evaluate returns
        straddle_return = abs(final_price - current_price) - straddle_price
        protective_put_strike = current_price - 2 * row["Realized Volatility"] * current_price
        protective_put_return = max(protective_put_strike - final_price, final_price) - protective_put_price
        buy_and_hold_return = final_price - initial_price

        # Append results
        results.append({
            "Column": col,
            "Straddle Return": straddle_return,
            "Protective Put Return": protective_put_return,
            "Buy-and-Hold Return": buy_and_hold_return
        })

    return pd.DataFrame(results)

def simulate_and_evaluate(df, num_simulations=1000):
    """
    Simulate test datasets, evaluate positions, and compute average returns.

    Parameters:
        df (pd.DataFrame): Original dataset used to derive parameters.
        num_simulations (int): Number of simulations to run.

    Returns:
        pd.DataFrame: Average relative returns for each asset-position pair.
    """
    initial_price = 100
    r = 0  # Risk-free rate
    T = 1  # Time horizon in years

    # Calculate prices for the original dataset
    prices_df = calculate_prices(df)

    # Initialize storage for cumulative returns
    cumulative_returns = {col: {"Straddle": 0, "Protective Put": 0, "Buy-and-Hold": 0} for col in df.columns}

    for _ in range(num_simulations):
        simulated_test_data = {}

        # Simulate test datasets based on original parameters
        for col in df.columns:
            log_returns = df[col].dropna()
            sigma = log_returns.std()
            simulated_returns = np.random.normal(loc=log_returns.mean(), scale=sigma, size=len(log_returns))
            simulated_test_data[col] = simulated_returns

        test_df = pd.DataFrame(simulated_test_data)

        # Evaluate positions on simulated data
        returns_df = evaluate_positions(test_df, prices_df)

        # Accumulate returns
        for _, row in returns_df.iterrows():
            col = row["Column"]
            cumulative_returns[col]["Straddle"] += row["Straddle Return"]
            cumulative_returns[col]["Protective Put"] += row["Protective Put Return"]
            cumulative_returns[col]["Buy-and-Hold"] += row["Buy-and-Hold Return"]

    # Compute average relative returns
    avg_returns = []
    for col, returns in cumulative_returns.items():
        avg_returns.append({
            "Column": col,
            "Avg Straddle Return": returns["Straddle"] / num_simulations,
            "Avg Protective Put Return": returns["Protective Put"] / num_simulations,
            "Avg Buy-and-Hold Return": returns["Buy-and-Hold"] / num_simulations
        })

    return pd.DataFrame(avg_returns)

# Example usage
# Assuming `df` is a DataFrame where each column represents daily log returns for 1 year
# avg_returns_df = simulate_and_evaluate(df, num_simulations=1000)
# print(avg_returns_df)


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import kurtosis, skew, shapiro, anderson, probplot

# Load datasets
files = ["gaussian_data.csv", "skew_normal_positive.csv", "skew_normal_negative.csv", 
         "skew_student_positive.csv", "skew_student_negative.csv"]

dataframes = {file: pd.read_csv(file)["Log_Return"] for file in files}

# Analysis Functions
def analyze_distribution(data, name):
    print(f"--- Analyzing {name} ---")
    
    # Normality Tests
    shapiro_stat, shapiro_p = shapiro(data)
    print(f"Shapiro-Wilk Test: Statistic={shapiro_stat:.4f}, p-value={shapiro_p:.4f}")
    
    ad_result = anderson(data)
    print(f"Anderson-Darling Test: Statistic={ad_result.statistic:.4f}, Critical Values={ad_result.critical_values}")
    
    # Kurtosis and Skewness
    k = kurtosis(data, fisher=True)  # Fisher=True means excess kurtosis
    s = skew(data)
    print(f"Kurtosis (Excess): {k:.4f} {'(Heavy tails)' if k > 3 else '(Thin tails)'}")
    print(f"Skewness: {s:.4f} {'(Positive skew)' if s > 0 else '(Negative skew)' if s < 0 else '(Symmetric)'}")
    
    # QQ Plot
    plt.figure(figsize=(8, 6))
    probplot(data, dist="norm", plot=plt)
    plt.title(f"QQ Plot for {name}")
    plt.grid()
    plt.show()

# Perform analysis for each dataset
for file, data in dataframes.items():
    analyze_distribution(data, file.split(".")[0])

print("Analysis completed.")
